In [2]:
import os
import cv2
import albumentations as A
import random
import albumentations as A
from albumentations import HorizontalFlip, Rotate, RandomBrightnessContrast, GaussianBlur
from albumentations import ShiftScaleRotate
from albumentations import Resize


# 输入目录和输出目录
input_dir = r"C:\Users\zhou\Desktop\dataset\train"
output_dir = r"C:\Users\zhou\Desktop\dataset\train(augmented)"

# 定义每个类别目标数量
target_counts = {
    "cyst": 4063,
    "stone": 4063,
    "tumor": 4063,
    "normal": None  # normal 不需要增强
}

# 确保输出目录存在
os.makedirs(output_dir, exist_ok=True)

# 定义增强函数
def augment_image(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussianBlur(blur_limit=(3, 5), p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5)
    ])
    return transform(image=image)['image']

# 对某个类别的图像进行增强
def augment_category(input_dir, output_dir, category, target_count):
    input_path = os.path.join(input_dir, category)
    output_path = os.path.join(output_dir, category)
    os.makedirs(output_path, exist_ok=True)
    
    # 获取类别中的所有图像
    image_files = [f for f in os.listdir(input_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    current_count = len(image_files)
    
    # 如果当前数量已经达到或超过目标数量，不需要增强
    if current_count >= target_count:
        print(f"{category} 已达到目标数量，无需增强。")
        return
    
    # 增强所需的数量
    augment_count = target_count - current_count
    print(f"{category}: 当前数量 {current_count}, 目标数量 {target_count}, 需要增强 {augment_count} 张。")
    
    for i in range(augment_count):
        # 随机选择一张原始图像
        random_image = random.choice(image_files)
        img_path = os.path.join(input_path, random_image)
        
        # 读取图像
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 转换为 RGB
        
        # 数据增强
        augmented_image = augment_image(image)
        
        # 保存增强后的图像
        save_path = os.path.join(output_path, f"aug_{i+1}_{random_image}")
        cv2.imwrite(save_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

# 对所有类别进行处理
for category, target_count in target_counts.items():
    if target_count:
        augment_category(input_dir, output_dir, category, target_count)

print("所有增强任务完成，增强后的图像保存在:", output_dir)


cyst: 当前数量 2969, 目标数量 4063, 需要增强 1094 张。
stone: 当前数量 1103, 目标数量 4063, 需要增强 2960 张。
tumor: 当前数量 1827, 目标数量 4063, 需要增强 2236 张。
所有增强任务完成，增强后的图像保存在: C:\Users\zhou\Desktop\dataset\train(augmented)
